In [ ]:
using TORA
using MeshCat
using RigidBodyDynamics

In [ ]:
TORA.greet()

In [ ]:
vis = Visualizer()

# Change the camera's Field of View
setprop!(vis["/Cameras/default/rotated/<object>"], "fov", 40)

# IJuliaCell(vis)  # Show the viewer here (in the notebook)
open(vis)  # Show the viewer in a separate tab

In [ ]:
robot = TORA.create_robot_franka("panda_arm", vis)
problem = TORA.Problem(robot, 201, 1/100)

# Constrain initial and final joint velocities to zero
TORA.fix_joint_velocities!(problem, robot, 1, zeros(robot.n_v))
TORA.fix_joint_velocities!(problem, robot, problem.num_knots, zeros(robot.n_v))

let CubicTimeScaling(Tf::Number, t::Number) = 3(t / Tf)^2 - 2(t / Tf)^3
    for k = 1:2:problem.num_knots  # For every other knot
        θ = CubicTimeScaling(problem.num_knots - 1, k - 1) * 2π
        pos = [0.5, 0.2 * cos(θ), 0.8 + 0.2 * sin(θ)]
        TORA.constrain_ee_position!(problem, k, pos)
    end
end

In [ ]:
TORA.show_problem_info(problem)

In [ ]:
initial_q = [0, 0, 0, -π/2, 0, π, 0]

zero!(robot.state)
set_configuration!(robot.state, initial_q)
set_configuration!(robot.mvis, configuration(robot.state))

initial_qs = repeat(initial_q, 1, problem.num_knots)
initial_vs = zeros(robot.n_v, problem.num_knots)
initial_τs = zeros(robot.n_τ, problem.num_knots)

initial_guess = [initial_qs; initial_vs; initial_τs]

# Flatten matrix and truncate torques of last knot
initial_guess = vec(initial_guess)[1:end - robot.n_τ];

In [ ]:
# Calling this will start the optimization
cpu_time, x, solver_log = TORA.solve_with_ipopt(problem, robot, initial_guess=initial_guess)

In [ ]:
TORA.play_trajectory(vis, problem, robot, x)

In [ ]:
TORA.plot_results(problem, robot, x)

In [ ]:
TORA.plot_log(solver_log)